In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.stats as sp
import scipy.optimize as sco
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)

In [2]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

def bisection(f, a, b, tol=1e-6):
    c = (a+b)/2
    while(np.abs(b-a)>tol):
        if f(c) == 0:
            return c
        elif f(a)*f(c)<0:
            b=c
        else:
            a=c
        c = (a+b)/2
    return c

def impliedVol(S0, K, tau, r, price, opt_type='c'):
    equation = lambda sigma: BSM(S0,K,tau,r,sigma,opt_type) - price
    return bisection(equation,-1,5)

In [25]:
files = ['20180427', '20180504', '20180511', '20180518', '20180525', '20180601', '20180608', '20180615',
        '20180720', '20180817', '20180921', '20181019', '20181116', '20190118', '20190215', '20190621',
        '20200117']
r = 0.0184 # 3m Treasury 4/30/18
dfs = []
for f in files:
    temp = pd.read_csv('{}.csv'.format(f))
    dfs.append(temp)
dfs

[     Unnamed: 0  Unnamed: 0.1         T       K   P_bid   P_ask   C_bid  \
 0             0             0  20180427   910.0    0.00    0.02  607.10   
 1             1             1  20180427   920.0    0.00    0.03  596.50   
 2             2             2  20180427   930.0    0.00    3.80  586.70   
 3             3             3  20180427   940.0    0.00    0.05  576.50   
 4             4             4  20180427   950.0    0.00    0.05  566.50   
 5             5             5  20180427   960.0    0.00    1.67  556.40   
 6             6             6  20180427   970.0    0.00    3.35  546.50   
 7             7             7  20180427   980.0    0.00    4.25  536.50   
 8             8             8  20180427   990.0    0.00    0.68  526.55   
 9             9             9  20180427  1000.0    0.00    0.04  516.55   
 10           10            10  20180427  1010.0    0.00    0.68  506.55   
 11           11            11  20180427  1020.0    0.00    0.08  496.75   
 12         

In [4]:
# import os
# os.chdir('/Users/Brendon/Downloads/AMZN')
# comb_dfs = []
# for f in files:
#     temp = pd.read_csv('{}vol.csv'.format(f))
#     comb_dfs.append(temp)
    
# for i in range(len(dfs)):
#     dfs[i]['Comb_vol'] = comb_dfs[i]['CombinedVol']
    
# dfs

[     Unnamed: 0         T       K   P_bid   P_ask   C_bid   C_ask        S  \
 0             1  20180427   910.0    0.00    0.02  607.10  611.80  1517.96   
 1             2  20180427   920.0    0.00    0.03  596.50  601.50  1517.96   
 2             3  20180427   930.0    0.00    3.80  586.70  591.70  1517.96   
 3             4  20180427   940.0    0.00    0.05  576.50  581.50  1517.96   
 4             5  20180427   950.0    0.00    0.05  566.50  571.50  1517.96   
 5             6  20180427   960.0    0.00    1.67  556.40  561.40  1517.96   
 6             7  20180427   970.0    0.00    3.35  546.50  551.50  1517.96   
 7             8  20180427   980.0    0.00    4.25  536.50  541.50  1517.96   
 8             9  20180427   990.0    0.00    0.68  526.55  531.55  1517.96   
 9            10  20180427  1000.0    0.00    0.04  516.55  521.55  1517.96   
 10           11  20180427  1010.0    0.00    0.68  506.55  511.55  1517.96   
 11           12  20180427  1020.0    0.00    0.08  

In [24]:
# os.chdir('/Users/Brendon/Documents/FE 800/AMZN')
# for i in range(len(dfs)):
#     dfs[i].to_csv('{}.csv'.format(files[i]))

In [4]:
# mid1s = [] # mid price
# mid2s = [] # mid IV
# for df in dfs:
#     temp = pd.DataFrame({'K':df['K'], 'TTM':df['TTM'], 'C_IV': df['C_IV'], 'P_IV': df['P_IV']})
#     temp['C_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, x.C_IV), axis=1)
#     temp['P_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, x.P_IV, 'p'), axis=1)
#     mid1s.append(temp)
#     temp = pd.DataFrame({'K':df['K'], 'TTM':df['TTM'], 'C_IV': (df['C_bid_IV'] + df['C_ask_IV'])/2, 'P_IV': (df['P_bid_IV'] + df['P_ask_IV'])/2})
#     temp['C_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, (x['C_bid_IV'] + x['C_ask_IV'])/2), axis=1)
#     temp['P_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, (x['P_bid_IV'] + x['P_ask_IV'])/2, 'p'),axis=1)
#     mid2s.append(temp)
    
# mid1s

[          K      TTM      C_IV          P_IV  C_BS_Price  P_BS_Price
 0     910.0  0.00274  4.495669  2.792831e+00  609.449999    0.010000
 1     920.0  0.00274  4.191422  2.813418e+00  599.000000    0.015000
 2     930.0  0.00274  4.211310  4.499743e+00  589.200000    1.900001
 3     940.0  0.00274  4.028153  2.801088e+00  579.000000    0.025000
 4     950.0  0.00274  3.947705  2.742732e+00  569.000000    0.025000
 5     960.0  0.00274  3.814493  3.782355e+00  558.900001    0.835000
 6     970.0  0.00274  3.789080  4.077714e+00  549.000000    1.675000
 7     980.0  0.00274  3.710865  4.141626e+00  539.000000    2.125000
 8     990.0  0.00274  3.657488  3.191054e+00  529.050000    0.340000
 9    1000.0  0.00274  3.580253  2.418402e+00  519.050000    0.020000
 10   1010.0  0.00274  3.503692  3.054464e+00  509.050000    0.340000
 11   1020.0  0.00274  3.512627  2.438032e+00  499.250000    0.040000
 12   1030.0  0.00274  3.435856  2.905584e+00  489.249999    0.325000
 13   1040.0  0.0027

In [13]:
tol = 10E-9
def SVI(df,a,b,p,m,sig):
    k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
    return a + b*(p*(np.asarray(k)-m) + np.sqrt((np.asarray(k)-m)**2 + sig**2))
def SVI_fit(df,IV_col):
    obj = lambda x: np.sum((SVI(df, x[0], x[1], x[2], x[3], x[4]) - np.asarray(df[IV_col]))**2)
    cons = ({'type':'ineq','fun': lambda x: x[0] + x[1]*x[4]*np.sqrt(1 - x[2]**2)},
            {'type':'ineq','fun': lambda x: x[1]},
            {'type':'ineq','fun': lambda x: x[4] - tol})
    bnds = ((None,None),(0,None),(-1+tol,1-tol),(None,None),(tol,None))
    guess = [.01,.01,.01,.01,.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    return result

SVI_fit(dfs[2],'Comb_vol')

     fun: 0.08929287211583578
     jac: array([-0.15156823, -0.07880143,  1.17131854, -0.12561405, -1.03094237])
 message: 'Iteration limit exceeded'
    nfev: 752
     nit: 101
    njev: 101
  status: 9
 success: False
       x: array([-4.60025714, 10.10504265,  0.65381079,  0.66036763,  0.65452857])

20180511

In [19]:
temp = SVI_fit(dfs[2],'Comb_vol')['x']
trace0 = go.Scatter(
    x = np.log(np.asarray(dfs[2].K)/(1517.96*np.exp(r*np.asarray(dfs[2].TTM)))), y = dfs[2].Comb_vol, 
    name = 'Combined IV'
)
trace1 = go.Scatter(
    x = np.log(np.asarray(dfs[2].K)/(1517.96*np.exp(r*np.asarray(dfs[2].TTM)))), 
    y = SVI(dfs[2],temp[0],temp[1],temp[2],temp[3],temp[4]), name = 'Raw SVI Fit'
)
data = [trace0,trace1]
layout = go.Layout(
    title = 'AMZN Call Raw SVI Date: {}'.format(dfs[2]['T'][0]), 
        yaxis = dict(title = 'Implied Volatility (%)'), xaxis = dict(title = 'Normalized Strike'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [22]:
# temp = SVI_fit(dfs[2],'Comb_vol')['x']
trace0 = go.Scatter(
    x = np.log(np.asarray(dfs[0].K)/(1517.96*np.exp(r*np.asarray(dfs[0].TTM)))), y = dfs[0].Comb_vol, 
    name = 'Combined IV {}'.format(dfs[0]['T'][0])
)
trace1 = go.Scatter(
    x = np.log(np.asarray(dfs[0].K)/(1517.96*np.exp(r*np.asarray(dfs[0].TTM)))), y = dfs[0].C_IV, 
    name = 'Call IV {}'.format(dfs[0]['T'][0])
)
trace2 = go.Scatter(
    x = np.log(np.asarray(dfs[1].K)/(1517.96*np.exp(r*np.asarray(dfs[1].TTM)))), y = dfs[1].Comb_vol, 
    name = 'Combined IV {}'.format(dfs[1]['T'][1])
)
trace3 = go.Scatter(
    x = np.log(np.asarray(dfs[1].K)/(1517.96*np.exp(r*np.asarray(dfs[1].TTM)))), y = dfs[1].C_IV, 
    name = 'Call IV {}'.format(dfs[1]['T'][1])
)
# trace1 = go.Scatter(
#     x = np.log(np.asarray(dfs[2].K)/(1517.96*np.exp(r*np.asarray(dfs[2].TTM)))), 
#     y = SVI(dfs[2],temp[0],temp[1],temp[2],temp[3],temp[4]), name = 'Raw SVI Fit'
# )
data = [trace0,trace1,trace2,trace3]
layout = go.Layout(
    title = 'AMZN Call Raw SVI Date: {} and {}'.format(dfs[0]['T'][0], dfs[1]['T'][0]), 
        yaxis = dict(title = 'Implied Volatility (%)'), xaxis = dict(title = 'Normalized Strike'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [23]:
for i in range(2,len(dfs)):
    temp = SVI_fit(dfs[i],'Comb_vol')['x']
    trace0 = go.Scatter(
        x = np.log(np.asarray(dfs[i].K)/(1517.96*np.exp(r*np.asarray(dfs[i].TTM)))), y = dfs[i].Comb_vol, 
        name = 'Call IV'
    )
    trace1 = go.Scatter(
        x = np.log(np.asarray(dfs[i].K)/(1517.96*np.exp(r*np.asarray(dfs[i].TTM)))), 
        y = SVI(dfs[i],temp[0],temp[1],temp[2],temp[3],temp[4]), name = 'Raw SVI Fit'
    )
    data = [trace0,trace1]
    layout = go.Layout(
        title = 'AMZN Combined Volatility Raw SVI Date: {}'.format(files[i]), 
            yaxis = dict(title = 'Implied Volatility (%)'), xaxis = dict(title = 'Normalized Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [19]:
mid1s[0].head()

,K,TTM,C_IV,P_IV,C_BS_Price,P_BS_Price
0,910.0,0.00274,4.495669,2.792831,609.449999,0.010000
1,920.0,0.00274,4.191422,2.813418,599.000000,0.015000
2,930.0,0.00274,4.211310,4.499743,589.200000,1.900001
3,940.0,0.00274,4.028153,2.801088,579.000000,0.025000
4,950.0,0.00274,3.947705,2.742732,569.000000,0.025000


In [20]:
tol = 10E-9
# def QE_SVI(df,a,b,p,m,sig):
#     k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
#     y = (k - m)/sig
#     return a + b*(p*(np.asarray(k)-m) + np.sqrt((np.asarray(k)-m)**2 + sig**2))

def QE_SVI(df,IV_col,m,sig):
    k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
    y = (k - m)/sig
    T = df.TTM[0]

    obj = lambda x: np.sum((x[2] + x[1]*y + x[0]*(np.sqrt(1+y**2)) - df[IV_col]*T)**2)
    cons = ({'type':'ineq','fun': lambda x: x[0] - np.abs(x[1])},
           {'type':'ineq','fun': lambda x: 4*sig - x[0] - np.abs(x[1])})
    bnds = ((0,4*sig),(None,None),(0,np.max(df[IV_col]*T)))
    guess = [0.01,0.01,0.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    
    b = result.x[0]/(sig*T)
    p = result.x[1]/(b*sig*T)
    a = result.x[2]/T
    return a + b*(p*(np.asarray(k)-m) + np.sqrt((np.asarray(k)-m)**2 + sig**2))
        

def QE_SVI_fit(df,IV_col):
    obj = lambda x: np.sum(np.sqrt((QE_SVI(df, IV_col, x[0], x[1]) - np.asarray(df[IV_col]))**2))
    cons = ({'type':'ineq','fun': lambda x: x[1]})
    bnds = ((None,None),(0,None))
    guess = [.01,.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    return result

QE_SVI_fit(mid1s[0],'C_IV')

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning:

divide by zero encountered in true_divide

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in multiply

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in add

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in double_scalars



     fun: 139.12896458259596
     jac: array([1298.16247177, 1604.97933197])
 message: 'Optimization terminated successfully.'
    nfev: 83
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([ 5.49546692, 36.13670511])

In [22]:
for i in range(len(mid1s)):
    temp = QE_SVI_fit(mid1s[i],'C_IV')['x']
    trace0 = go.Scatter(
        x = np.log(np.asarray(mid1s[i].K)/(1517.96*np.exp(r*np.asarray(mid1s[i].TTM)))), y = mid1s[i].C_IV, 
        name = 'Call IV'
    )
    trace1 = go.Scatter(
        x = np.log(np.asarray(mid1s[i].K)/(1517.96*np.exp(r*np.asarray(mid1s[i].TTM)))), 
        y = QE_SVI(mid1s[i],'C_IV',temp[0],temp[1]), name = 'QE-SVI Fit'
    )
    data = [trace0,trace1]
    layout = go.Layout(
        title = 'AMZN Call QE-SVI Date: {}'.format(files[i]), 
            yaxis = dict(title = 'Implied Volatility (%)'), xaxis = dict(title = 'Normalized Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning:

divide by zero encountered in true_divide

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in multiply

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in add

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in double_scalars

